In [1]:
!pip install -q -U google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 6.8 MB/s eta 0:00:00


In [ ]:
!git clone --branch main https://github.com/giankev/Ancient-to-Modern-Italian-Automatic-Translation.git

In [7]:
from kaggle_secrets import UserSecretsClient
import huggingface_hub
import os


try:
    user_secrets = UserSecretsClient()
    API_KEY = user_secrets.get_secret("api_google_ai")
    print("API KEY Loaded")
except Exception as e:
    print(f"Error during programmatic login: {e}")
    print("Please ensure your HF_TOKEN secret is correctly set in Kaggle.")

API KEY Loaded


In [20]:
from time import sleep

In [ ]:
import pandas as pd

csv_translation = "/kaggle/working/Ancient-to-Modern-Italian-Automatic-Translation/Gemma2b-it-translation/CulturalIA-hw2_transl-Gemma2b-it_context_learning.csv"
df = pd.read_csv(csv_translation)

In [10]:
df.head()

,old_text,translation
0,quella guerra ben fatta l' opera perché etc. E...,"""Quella guerra ben fatta, l'opera perché etc. ..."
1,"crudele, e di tutte le colpe pigli vendetta, c...","""crudele, e di tutte le colpe si prende vendet..."
2,Non d' altra forza d' animo fue ornato Ponzio ...,"""Non ha altra forza d'animo che l'onore di Pon..."
3,Se questo piace a tutti e se 'l tempo hae biso...,"""Se questo piace a tutti e se il tempo ha biso..."
4,Officio di questa arte pare che sia dicere app...,"""L'arte di questa disciplina sembra essere que..."


In [ ]:
from google import genai

client = genai.Client(api_key=API_KEY)

In [16]:
def prompt_evaluation(original: str, translated: str):
    prompt =  f"""Valuta la seguente traduzione dall'italiano antico all'italiano moderno.
                Concentrati sulla fedeltà semantica e la correttezza grammaticale.
                Fornisci un punteggio da 1 a 5 basandoti sulla seguente rubrica:
                1. Traduzione completamente inaccettabile: la traduzione non ha pertinenza con il significato originale, la frase generata è incomprensibile (gibberish) o senza senso.
                2. Errori semantici gravi, omissioni o aggiunte sostanziali rispetto alla frase originale. Gli errori sono di natura semantica e sintattica. È qualcosa che un umano non scriverebbe mai.
                3. Traduzione parzialmente errata: la traduzione è mediocre, contiene errori, ma sono per lo più errori minori, come refusi o piccoli errori semantici.
                4. Buona traduzione: la traduzione è per lo più corretta, sostanzialmente fedele al testo originale, ma lo stile potrebbe non corrispondere perfettamente alla frase originale; è comunque fluente, comprensibile e semanticamente accettabile.
                5. Traduzione perfetta: la traduzione è accurata, fluente, completa e coerente. Ha mantenuto il significato originale il più possibile.
                fornisci solo il numero del punteggio (da 1 a 5).
                frase originale: "{original}"
                frase tradotta: "{translated}"
                
                """
    return prompt

In [26]:
results = []
for index, row in df.iterrows():
    
    original = row["old_text"]
    translation = row["translation"]
    prompt = prompt_evaluation(original, translation)

    response = client.models.generate_content(
             model="gemini-2.0-flash-lite",
             contents=prompt
    )

    results.append({
        "sentence": original,
        "translation": translation,
        "score": response.text
    })

    sleep(2) 

In [34]:
df_results = pd.DataFrame(results)
df_results.to_csv("/kaggle/working/evaluation_gemma2b-it_context_learning.csv", index=False, encoding='utf-8')

In [35]:
import json

with open("/kaggle/working/evaluation_gemma2b-it_context_learning.jsonl", "w", encoding="utf-8") as f:
    for entry in results:
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')